# Linear Regression with Torch

Toy example of linear regression model made with Torch. Three main components of the model are:
 - **nn.Linear** - linear layer of neural network that performs (mx + b) operation
 - **nn.MSELoss** - Mean Squared Loss function
 - **torch.optim.SGD** - Stochastic Gradient Descend

In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_boston
import pandas as pd

## 1. Model

In [2]:
class TorchLinearRegression:
    
    def __init__(self, lr=0.001):
        # learning rate
        self.lr = lr
                
    def fit(self, X, y, epochs=60):
        # array to tensor
        X = torch.from_numpy(X).type(torch.float32)
        y = torch.from_numpy(y).type(torch.float32)
        # model parameters
        input_size = X.shape[1]
        output_size = y.shape[1]
        # model, loss, optimizer
        self.model = nn.Linear(input_size, output_size)
        criterion = nn.MSELoss()
        optimizer = torch.optim.SGD(self.model.parameters(), lr=self.lr) 
        # training
        for i in range(epochs):
            # forward
            outputs = self.model(X)
            # compute loss
            loss = criterion(outputs, y)
            # set gradients to 0
            optimizer.zero_grad()
            # backward
            loss.backward()
            # update parameters
            optimizer.step()
            
    def predict(self, X):
        X = torch.from_numpy(X).type(torch.float32)
        y_pred = self.model(X).detach().numpy()
        return y_pred

## 2. Data

Features scaled to [0, 1]

In [3]:
df = pd.DataFrame(load_boston()['data'], columns=load_boston()['feature_names'])
df['target'] = load_boston()['target']
data = MinMaxScaler().fit_transform(df)

X = data[:, :-1] # all the feautres
y = data[:, -1:] # target

df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


## 3. Training

In [4]:
model = TorchLinearRegression(lr=0.1)
model.fit(X, y, 500)
y_pred = model.predict(X)

## 4. Evaluation

In [5]:
mean_squared_error(y, y_pred)

0.011920771594821645

In [6]:
m = nn.Linear(1, 1)